In [1]:
# initializing otter-grader
import otter
grader = otter.Notebook()

# Lab 9: Putting it all Together

In this lab assignment we will pick bits and pieces of what a project would look like. This is not a full project, but it covers the following ideas and tries to link them to how to analyze a dataset:
* Visualization and Exploratory Data Analysis (EDA)
* Multilinear regression and dealing with missing information
* Correlation
* Bonus: PCA

This can be considered practice for your final project, but your project should not be as simple as this lab and should analyze in more depth than we can cover here.

## Due Date
This assignment is due **Friday, May 29 at 11:59pm PST.**

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.



**Collaborators**: ...

In [2]:
import numpy as np
import pandas as pd
import altair as alt
import scipy
import sklearn
from sklearn.linear_model import LinearRegression

## Life Expectancy Data

This data was taken from [this source](https://www.kaggle.com/kumarajarshi/life-expectancy-who), and the goal of this dataset is to see what factors are contributors to life expectancy.

In [3]:
df = pd.read_csv('life_expectancy.csv')
df.head()

Country  Year      Status  Life Expectancy  Adult Mortality  \
0  Afghanistan  2015  Developing             65.0            263.0   
1  Afghanistan  2014  Developing             59.9            271.0   
2  Afghanistan  2013  Developing             59.9            268.0   
3  Afghanistan  2012  Developing             59.5            272.0   
4  Afghanistan  2011  Developing             59.2            275.0   

   Infant Deaths  Alcohol  Percentage Expenditure  Hepatitis B  Measles  ...  \
0             62     0.01               71.279624         65.0     1154  ...   
1             64     0.01               73.523582         62.0      492  ...   
2             66     0.01               73.219243         64.0      430  ...   
3             69     0.01               78.184215         67.0     2787  ...   
4             71     0.01                7.097109         68.0     3013  ...   

   Polio  Total expenditure  Diphtheria  HIV/AIDS         GDP  Population  \
0    6.0               8.16        65.0       0.1  584.259210  33736494.0   
1   58.0               8.18        62.0       0.1  612.696514    327582.0   
2   62.0               8.13        64.0       0.1  631.744976  31731688.0   
3   67.0               8.52        67.0       0.1  669.959000   3696958.0   
4   68.0               7.87        68.0       0.1   63.537231   2978599.0   

   Thinness 1-19 Years  Thinness 5-9 Years  Income composition of resources  \
0                 17.2                17.3                            0.479   
1                 17.5                17.5                            0.476   
2                 17.7                17.7                            0.470   
3                 17.9                18.0                            0.463   
4                 18.2                18.2                            0.454   

   Schooling  
0       10.1  
1       10.0  
2        9.9  
3        9.8  
4        9.5  

[5 rows x 22 columns]

In today's lab we will aim to answer these three questions:
1. What is the general trend of life expectancy over time?
2. What is a multilinear regression model that can predict life expectancy, and how accurate is it?
3. Is life expectancy a better indicator of how long people live, or the childhood mortality rate in developing nations?

## Question 0
Before we can begin with data analysis, we have to do a little bit of data exploration. Let's start familiarizing outselves with the data by asking the basic questions:
1. Are there any missing values? If so, how many?
2. Are there any unique values? Any values that might seem strange?
3. Are there any outliers? How does the distribution of the data change if you remove them?

### Question 0a
In the cell box below, include your code for investigating these questions. As with any data exploration, there is more than one way to do approach these questions. There is no correct answer for this, so you will get points for clear, commented code.
<!--
BEGIN QUESTION
name: q0a
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

In [18]:
## ADD DATA EXPLORATION CODE HERE.
## using shape to see number of columns and observations, and describe to get basic stats for each column
df.shape
df.describe()

## using isnull to see if there are any missing values and get the counts for each column
df.isnull().sum()

## using unique function to find unique values in numerical to see if we can catch any outliers
list(df['Life Expectancy'].unique())
list(df['Adult Mortality'].unique())
list(df['Infant Deaths'].unique())

## using unique function for status and country name to find any potential typos/unique observations
list(df['Country'].unique())
list(df['Status'].unique())

## looking for outliers
df.loc[df['Life Expectancy'] > 85]
df.loc[df['Infant Deaths'] > 500]
df.loc[df['Adult Mortality'] > 300]


Country  Year  Status  Life Expectancy  Adult Mortality  \
14    Afghanistan  2001       0             55.3            316.0   
15    Afghanistan  2000       0             54.8            321.0   
48         Angola  2015       0             52.4            335.0   
49         Angola  2014       0             51.7            348.0   
50         Angola  2013       0             51.1            355.0   
...           ...   ...     ...              ...              ...   
2932     Zimbabwe  2005       0             44.6            717.0   
2933     Zimbabwe  2004       0             44.3            723.0   
2934     Zimbabwe  2003       0             44.5            715.0   
2936     Zimbabwe  2001       0             45.3            686.0   
2937     Zimbabwe  2000       0             46.0            665.0   

      Infant Deaths  Alcohol  Percentage Expenditure  Hepatitis B  Measles  \
14               88     0.01               10.574728         63.0     8762   
15               88     0.01               10.424960         62.0     6532   
48               66      NaN                0.000000         64.0      118   
49               67     8.33               23.965612         64.0    11699   
50               69     8.10               35.958573         77.0     8523   
...             ...      ...                     ...          ...      ...   
2932             28     4.14                8.717409         65.0      420   
2933             27     4.36                0.000000         68.0       31   
2934             26     4.06                0.000000          7.0      998   
2936             25     1.72                0.000000         76.0      529   
2937             24     1.68                0.000000         79.0     1483   

      ...  Polio  Total expenditure  Diphtheria  HIV/AIDS          GDP  \
14    ...   35.0               7.80        33.0       0.1   117.496980   
15    ...   24.0               8.20        24.0       0.1   114.560000   
48    ...    7.0                NaN        64.0       1.9  3695.793748   
49    ...   68.0               3.31        64.0       2.0   479.312240   
50    ...   67.0               4.26        77.0       2.3   484.616884   
...   ...    ...                ...         ...       ...          ...   
2932  ...   69.0               6.44        68.0      30.3   444.765750   
2933  ...   67.0               7.13        65.0      33.6   454.366654   
2934  ...    7.0               6.52        68.0      36.7   453.351155   
2936  ...   76.0               6.16        75.0      42.1   548.587312   
2937  ...   78.0               7.10        78.0      43.5   547.358879   

      Population  Thinness 1-19 Years  Thinness 5-9 Years  \
14     2966463.0                  2.1                 2.4   
15      293756.0                  2.3                 2.5   
48     2785935.0                  8.3                 8.2   
49     2692466.0                  8.5                 8.3   
50     2599834.0                  8.6                 8.5   
...          ...                  ...                 ...   
2932    129432.0                  9.0                 9.0   
2933  12777511.0                  9.4                 9.4   
2934  12633897.0                  9.8                 9.9   
2936  12366165.0                  1.6                 1.7   
2937  12222251.0                 11.0                11.2   

      Income composition of resources  Schooling  
14                              0.340        5.9  
15                              0.338        5.5  
48                              0.531       11.4  
49                              0.527       11.4  
50                              0.523       11.4  
...                               ...        ...  
2932                            0.406        9.3  
2933                            0.407        9.2  
2934                            0.418        9.5  
2936                            0.427        9.8  
2937                            0.434        9.8  

[360 rows

### Question 0b
Answer the three questions in the cell below.
For convenience:
1. Are there any missing values? If so, how many?
2. Are there any unique values? Any values that might seem strange?
3. Are there any outliers? How does the distribution of the data change if you remove them?
<!--
BEGIN QUESTION
name: q0a
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

There are 2563 missing values in the dataset, spread out across the 22 columns.

There don't appear to be any unique values that I could detect from the categorical variables.  I used unique on the numerical categories to find any potential outliers, which will be discussed in the following paragraph.

There are two countries that are outliers in infant deaths: Nigeria and India.  For 14/16 of its observations, they have infant deaths of over 1000.  Interestingly, the data specifies that the infant death rate is calculated per 1000 population. The next outlier for infant death rate is Nigeria.  They have multiple observations with infant deaths of over 500.  In terms of outliers for adult mortality, Zimbabwe has the largest outliers.  Looking at the output from df.describe, the 75th percentile for the column lies at about 228 deaths per 1000 in the population.  There are 360 observations with over 300 deaths.  Outliers for life expectancy, which I labeled as any country with a life expectancy of over 80, are mostly consisted of EU affiliated countries and a few Asian countries.  

## Question 1
Let's start with the easiest question. What is the general trend of life expectancy over time?


### Question 1a
Since there are so many countries, we should figure out a way to group countries together so that a plot we create is not cluttered to the point of having too much information. We are given one binary categorical variable called "Status", so let us look into grouping by that column. Create a new dataframe by grouping by "Status" and "Year" and taking the mean of the life expectancy for both categories. This dataframe should be indentical in structure to the original dataframe minus the "Country" column.

* Hint: Don't forget to `reset_index()`

<!--
BEGIN QUESTION
name: q1a
points: 2
manual: true
-->
<!-- EXPORT TO PDF -->

In [5]:
trend_df = df.groupby(['Status', 'Year']).mean()
trend_df = trend_df.reset_index()
trend_df.head()

Status  Year  Life Expectancy  Adult Mortality  Infant Deaths  \
0  Developed  2000        76.803125         91.71875        1.78125   
1  Developed  2001        77.128125         90.28125        1.75000   
2  Developed  2002        77.546875         86.46875        1.71875   
3  Developed  2003        77.940625         81.93750        1.65625   
4  Developed  2004        78.384375         84.46875        1.65625   

     Alcohol  Percentage Expenditure  Hepatitis B     Measles        BMI  ...  \
0   9.930625             1897.607997    71.470588   810.75000  45.581250  ...   
1   9.861250             2042.071819    75.500000   956.81250  48.950000  ...   
2  10.018125             1741.772286    86.578947  1531.50000  52.315625  ...   
3  10.126562             1532.335532    87.200000   701.15625  49.487500  ...   
4  10.188750             2962.759738    91.900000   331.43750  50.031250  ...   

      Polio  Total expenditure  Diphtheria  HIV/AIDS           GDP  \
0  89.06250           7.021562     83.6875       0.1  15215.825425   
1  93.87500           6.820625     92.0000       0.1  16036.874398   
2  91.93750           7.113750     92.2500       0.1  13724.055247   
3  94.96875           7.143125     92.8125       0.1  12449.282651   
4  92.31250           7.477188     95.1250       0.1  22409.726689   

     Population  Thinness 1-19 Years  Thinness 5-9 Years  \
0  4.342765e+06             1.462500            1.440625   
1  4.487960e+06             1.434375            1.412500   
2  7.345101e+06             1.418750            1.393750   
3  7.760736e+06             1.381250            1.368750   
4  8.355623e+06             1.362500            1.343750   

   Income composition of resources  Schooling  
0                         0.815345  14.982759  
1                         0.821138  15.124138  
2                         0.827034  15.317241  
3                         0.831862  15.431034  
4                         0.837966  15.662069  

[5 rows x 21 columns]

### Question 1b
Plot the year on the x-axis, mean life expectancy on the y-axis, and the binary value "Status" as the color. Be sure to set the scale of your axis so that the visualization makes sense.

<!--
BEGIN QUESTION
name: q1b
points: 2
manual: true
-->
<!-- EXPORT TO PDF -->

In [6]:
# CREATE YOUR PLOT BELOW
alt.Chart(trend_df, title = 'Mean Life Expectancy Comparison Between Developed and Developing Countries').mark_line(
).encode(
x = alt.X('Year', axis = alt.Axis(title = 'Year')),
y = alt.Y('Life Expectancy', axis = alt.Axis(title = 'Mean Life Expectancy'), scale=alt.Scale(domain=(64, 82))),
color = alt.Color('Status'))

alt.Chart(...)

### Question 1c
What can you conclude about our original question?

<!--
BEGIN QUESTION
name: q1c
points: 2
manual: true
-->
<!-- EXPORT TO PDF -->

Countries will increase their mean life expectancy over time, regardless of their status of development.  

## Question 2
Now let's ask the second question. What is the multilinear regression line that predicts life expectancy?

### Question 2a
In order to do this, let's convert the "Status" column into an indicator variable by converting "Developing" to 0 and "Developed" to 1. Then like we did in lab 8, calculate the multilinear regression and the **mean squared error** of the regression. You will learn more about analyzing the significance of each variable in this regression line in PSTAT 126.

Since you already did mean squared loss and adding bias in the previous lab, we have them provided for you. However, you have to create the rest of the regression.

In [7]:
def mean_squared_error(y, y_hat):
    return np.mean((y-y_hat)**2)

def add_bias(data):
    data.insert(0,'ones',1)
    return data

Before we do a linear regression, we need to clean the data. This is a very important step because this dataset, and many others that you will encounter will contain missing values. In addition to converting the "Status" column to an indicator, let's impute data with the mean of the column. While it would be better to impute values using a more representative value, for the interest of simplicity, we will just use the mean of each column.
* Hint: Look at how to use `fillna()`

<!--
BEGIN QUESTION
name: q2a
points: 3
manual: false
-->

In [8]:
# Replace the "Status" column with numerical values
df = df.replace({'Developed' : 1, 'Developing' : 0})
# Impute missing values
df_clean = df.fillna(df.mean())


<!--
BEGIN QUESTION
name: q2a2
points: 3
manual: false
-->

In [9]:
# Pick out the target value
y = df_clean['Life Expectancy']
# Remove "Year", "Country", and the target value from the dataframe
dfcols = list(df.columns.values)
dfcols.remove('Year')
dfcols.remove('Country')
dfcols.remove('Life Expectancy')
x = df_clean[dfcols]
# Fit a linear regression model using sklearn
model = LinearRegression()
model.fit(X = x, y = y)
# Get y_hat by predicting using the model
y_hat = model.predict(x)
loss = mean_squared_error(y,y_hat)
loss

16.289412396516717

### Question 2b

Let's take a look at the coefficients of our multilinear regression.

In [10]:
model.coef_

array([ 1.60009934e+00, -1.98736851e-02,  9.99038223e-02,  6.38886327e-02,
        8.92959806e-05, -1.47543045e-02, -1.93761604e-05,  4.43870010e-02,
       -7.48025311e-02,  2.86031840e-02,  6.40010641e-02,  4.01273813e-02,
       -4.69755732e-01,  3.28324464e-05,  2.57337357e-10, -8.22974803e-02,
        7.44473726e-03,  5.72691229e+00,  6.55472790e-01])

List 2 observations you notice from the regression coefficients and analyze them.

<!--
BEGIN QUESTION
name: q2b
points: 4
manual: true
-->
<!-- EXPORT TO PDF -->

One of the regression coefficients has a value of 2.5733e-10.  This implies that the effect the variable associated with this coefficient has on life expectancy is very small.

There are really only two variables in this dataset that appear to have a visible effect on life expectancy: the variable associated with regression coefficient 1.6 and the variable associated with regression coefficient 5.7269.

## Question 3

Now let us consider the last question. Is life expectancy a better indicator of child mortality, or of increasing the length of time people are expected to live in developing nations? 

### Question 3a

First filter the dataset to only consider developing nations (Remember that this is the value 0 now). Then find the $r^2$ values for the following columns:
* Infant Deaths
* Deaths under 5

Hint: use `corr()`

<!--
BEGIN QUESTION
name: q3a
points: 3
manual: false
-->

In [15]:
# First filter the dataset
df_developing = df_clean[df_clean["Status"] == 0]

# Calculate r^2 for "Infant deaths"
inf_deaths_r2 = (df_developing['Infant Deaths'].corr(df_developing['Life Expectancy']))**2
print(f'r^2 of Infant Deaths: {inf_deaths_r2}')
# Calculate r^2 for "Deaths under 5"
child_deaths_r2 = (df_developing['Deaths under 5'].corr(df_developing['Life Expectancy']))**2
print(f'r^2 of Deaths under 5: {child_deaths_r2}')

r^2 of Infant Deaths: 0.027789207385782947
r^2 of Deaths under 5: 0.038246585516551475


### Question 3b
What do these $r^2$ values tell you about the importance of childhood mortality statistics in predicting life expectancy?

<!--
BEGIN QUESTION
name: q3b
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

This isn't a very good model.  r^2 values close to 0 indicate that most of the variability of the responses around the mean isn't explained by the model.  

## BONUS: PCA

Using the same value of x as we used in question 2, what is the minimum number of principal components needed to run multilinear regression on to get a loss value within 10% of our original loss value when the data is centered and scaled? Feel free to use the two imports given below. 

Show all your work and get the correct number of principal components to get bonus points.

In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# SCRATCH WORK
...

Ellipsis

<!--
BEGIN QUESTION
name: bonus
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

# Running Built-in Tests
1. All tests are in `tests` directory
1. Each python file in `tests` is a test
1. `grader.check('testname')` runs test `'testname'`, e.g. `'q1'`
1. `grader.check_all()` runs all visible tests

In [17]:
# Run built-in checks
grader.check_all()

In [ ]:
# Generate pdf in classic notebook (does not work in JupyterLab)
import nb2pdf
nb2pdf.convert('lab09.ipynb')

# To generate pdf using command-line, run in terminal,
# nb2pdf lab09.ipynb

<IPython.core.display.Javascript object>

# Submission Checklist
1. Check filename is 'lab09.ipynb'
1. Save file to confirm all changes are on disk
1. Run *Kernel > Restart & Run All* to execute all code from top to bottom
1. Check `grader.check_all()` output
1. Save file again to write any new output to disk
1. Check generated pdf that all responses are displayed correctly
1. Submit to Gradescope